**Preparation and Exploratory Analysis**

Load libraries:


In [ ]:
rm(list = ls())

#These if statements used in case libraries are not installed on someone else's environment
if (!require(tidyverse)) install.packages('tidyverse')
if (!require(xts)) install.packages('xts')
if (!require(caret)) install.packages('caret')
if (!require(kernlab)) install.packages('kernlab')
if (!require(roll)) install.packages('roll')
if (!require(Rcpp)) install.packages('Rcpp')
if (!require(skimr)) install.packages('skimr')
if (!require(h2o)) install.packages('h2o')

library(tidyverse)
library(xts)
library(caret)
library(kernlab)
library(roll)
library(Rcpp)
library(skimr)
library(h2o)


Change path to the data on your computer:



In [ ]:
#data <- read.csv("C:/Users/d/Downloads/dfi_labs_BTCUSDT-1h-spot-data.csv")


data <- read.csv("/Users/nikhil/Downloads/dfi_labs_BTCUSDT-1h-spot-data.csv")


Data preparation:



In [ ]:
data$Start <- strptime(data$Start_time, format = "%M/%d/%Y %H:%M")



Volatility calculation:

Volatility was defined as the standard deviation of the opening prices for the previous 24 hours of a given trading window. I thought about using an API to pull the data for 05/31 but I didn't know which exchange was used in the rest of the data, and they are all a little different. I ended up deciding to take the standard deviation of the opening prices on 6/1 and make that the volatility value for the first 24 hours.


In [ ]:
#calculate rolling SD
volatility_rest <- roll_sd(data$Open, width = 24)

#remove the first 24 values
volatility_rest <- roll_sd(data$Open, width = 24)[24:5575]

#calculate SD of BTC price for 1st trading day
for_now <- sd(data$Open[1:24])

#since we don't have data before, use the first day's SD as the rolling SD for the first day
vec_holder <- rep(for_now, 24)

#add vectors together
volatility <- c(vec_holder, volatility_rest)

#create a new column in the DF to represent volatilty (sigma/rolling standard deviation of previous 24 hours BTC opening price)
data$volatility <- volatility



Stop loss/Take Profit Price calculations (using 2*sigma formula):


In [ ]:
#calculate take profit and stop loss price for each trading interval as Open +/- 2*sigma
data$take_profit <- data$Open + 2*data$volatility
data$stop_loss <- data$Open - 2*data$volatility

#create columns to indicate if BTC price hit stop/take profit price/remained in band within each interval
data$hit_stop <- ifelse(data$Low <= data$stop_loss, 1, 0)
data$hit_take_profit <- ifelse(data$High >= data$take_profit, 1, 0)
data$within_band <- ifelse(data$hit_stop == 0 & data$hit_take_profit == 0, 1, 0)

#there were 4 intervals where BTC price hit both stop and take profit, this column indicates this
data$hit_both_stop_and_profit <- ifelse(data$hit_stop == 1 & data$hit_take_profit == 1, 1, 0)


Calculate and visualize how many observations in the data set fall into each category. I added a fourth category because I noticed that there were some instances where the price hit both the stop loss and the take profit price.



In [ ]:
#Use the above columns to determine which of 4 categories each trading interval corresponded to. This will be the outcome variable once we start modeling.
data$category[data$hit_stop == 1 & data$hit_both_stop_and_profit == 0] <- "HIT_STOP"
data$category[data$hit_take_profit == 1 & data$hit_both_stop_and_profit == 0] <- "TAKE_PROFIT"
data$category[data$within_band == 1 & data$hit_both_stop_and_profit == 0] <- "WITHIN_BAND"
data$category[data$hit_stop == 1 & data$hit_both_stop_and_profit == 1] <- "HIT_BOTH"


#calculate column totals
tmp_vec <- sapply(data[14:17], sum)

#use column sums to figure out how many in each category, subtract out non mutually exclusive events
num_hit_both <- as.integer(tmp_vec[4])
num_hit_stop <- as.integer(tmp_vec[1]) - num_hit_both
num_hit_TP <- as.integer(tmp_vec[2]) - num_hit_both
num_within_band <- as.integer(tmp_vec[3])
total_observations <- count(data)$n

#turn into a df to graph with ggplot
tmp <- data.frame(
  "categories" <- c("hit_both", "hit_stop", "hit_TP", "stayed_in_band", "total_observations"),
  "values" <- c(num_hit_both, num_hit_stop, num_hit_TP, num_within_band, total_observations)
)
colnames(tmp) <- c("category", "value")


ggplot(data = tmp[1:4,], aes(x = category, y =value, fill = category)) + 
  geom_bar(stat = "identity") + 
  geom_text(aes(label=value), position=position_dodge(width=0.9), vjust=-0.25) +
  ggtitle("Number of Data Points by Category")


This column can be used to calculate the total profit that would be made using a hypothetical trading algorithm. The column value corresponds to the total difference in price between the entry and exit price of each trading window, based on the value of the "category column". To find the actual profit, create a new column that represents this column plus the amount of USD traded in a given trading window, then sum the column. This wasn't given so I did not calculate the actual number.



In [ ]:
data$profit <- NA

#Once we know how much is being traded per trading window, to find the actual number multiply below by the volume of BTC traded. Below is only calculating the difference in trade entry/exit prices.
for(i in 1:length(data$Open)){
  
  #If hits both stop and TP price, then consider the exit price of the trade to be avg of TP/Stop since you don't know which happened first
  if(data$category[i] == "HIT_BOTH"){
    data$profit[i] <- data$Open[i] - mean(data$take_profit[i], data$stop_loss[i])
  }else if(data$category[i] == "HIT_STOP"){
    data$profit[i] <- data$stop_loss[i] - data$Open[i]
  }else if(data$category[i] == "TAKE_PROFIT"){
    data$profit[i] <- data$stop_loss[i] + data$Open[i]
  }else{ #This is the within_band category but you don't need to specify because its the last option
   data$profit[i] <- data$Close[i] - data$Open[i] 
  }
}


Quick pairs plot to identify any obvious correlations between variables:  



In [ ]:
tmp <- select(data, category, qav, Open, volatility, profit)
pairs(tmp[2:5])



Generate summary stats of all variables in the data frame:


In [ ]:
skim(data)



Now lets prepare to make some models to predict which category the BTC price will fall into:

**Modeling**

Use an 80/20 split to split the data into testing and training sets to avoid model overfitting: 


In [ ]:
a <- createDataPartition(data$category, p = 0.8, list=FALSE)
training <- data[a,]
test <- data[-a,]


Fit a simple Linear Discriminant Analysis (LDA) model, evaluate what proportion of cases are predicted correctly:



In [ ]:
#fit model
lda <- train(category ~ qav + Open + volatility, 
                data = training, 
                method="lda")
#Generate predictions
test$predicted_category <- predict(lda, newdata = test)

num_test_observations <- count(test)$n

#Calculate the number of correct predictions
#make a binary column to represent if the prediction was correct.
test$predicted_correct <- ifelse(test$category == test$predicted_category, 1, 0)

num_correct_predictions <- sum(test$predicted_correct)
lda_test_prop_predicted_correctly <- num_correct_predictions/num_test_observations

lda_test_prop_predicted_correctly


Fit a Support Vector Machines (SVM) model: 



In [ ]:
# Set up Repeated k-fold Cross Validation
train_control <- trainControl(method="cv", number=10)

# Fit the model 
svm1 <- train(category ~ Open + volatility, data = data, method = "svmLinear", trControl = train_control)
#View the model
svm1


Make predictions using the SVM model:



In [ ]:
#on full data
data$predicted_category <- predict(svm1, newdata = data)

#Calculate the number of correct predictions
#make a binary column to represent if the prediction was correct.
data$predicted_correct <- ifelse(data$category == data$predicted_category, 1, 0)

num_correct_predictions <- sum(data$predicted_correct)
full_data_prop_predicted_correctly <- num_correct_predictions/total_observations

full_data_prop_predicted_correctly

#on test
test$predicted_category <- predict(svm1, newdata = test)

#Calculate the number of correct predictions
#make a binary column to represent if the prediction was correct.
test$predicted_correct <- ifelse(test$category == test$predicted_category, 1, 0)

test_num_correct_predictions <- sum(test$predicted_correct)
svm_1_test_prop_predicted_correctly <- test_num_correct_predictions/num_test_observations

svm_1_test_prop_predicted_correctly


**Add more variables with lag**

The previous models did not include qav or profit as variables. That is because qav and profit are both metrics that require information that is only obtained at the end of a trading window, and thus cannot be used to predict. I wanted to include these variables in the model, so I used a lag function to make the value of the qav and profit variables equal to their values in the previous trading window.


In [ ]:
#use indexing to shift the value of qav and profit down by one row
lagged_qav <- data$qav[1:5575]
lagged_profit <- data$profit[1:5575]
lagged_dataset <- data
lagged_dataset$qav[1] <- NA
lagged_dataset$profit[1] <- NA
lagged_dataset$qav[2:5576] <- lagged_qav
lagged_dataset$profit[2:5576] <- lagged_qav

#remove the first row of the dataset so that way NA value doesn't throw errors when fitting model
lagged_dataset <- lagged_dataset[2:5576,]


SVM using Open + Volatility + qav:



In [ ]:
# Set up Repeated k-fold Cross Validation
train_control <- trainControl(method="cv", number=5)

svm2 <- train(category ~ Open + volatility + qav, 
              data = lagged_dataset, method = "svmLinear", 
              trControl = train_control)

#View the model
svm2


#on test
test_observations <- count(test)$n
test$predicted_category <- predict(svm2, newdata = test)

#Calculate the number of correct predictions
#make a binary column to represent if the prediction was correct.
test$predicted_correct <- ifelse(test$category == test$predicted_category, 1, 0)

test_num_correct_predictions <- sum(test$predicted_correct)
svm_2_test_prop_predicted_correctly <- test_num_correct_predictions/test_observations

svm_2_test_prop_predicted_correctly


SVM using Open + Volatility + lagged qav + lagged profit:



In [ ]:
# Set up Repeated k-fold Cross Validation
train_control <- trainControl(method="cv", number=5)

svm3 <- train(category ~ Open + volatility + qav + profit, 
              data = lagged_dataset, method = "svmLinear", 
              trControl = train_control)

#View the model
svm3
data$svm_predicted_category <- predict(svm3, newdata = data)


#on test
test_observations <- count(test)$n
test$predicted_category <- predict(svm3, newdata = test)

#Calculate the number of correct predictions
#make a binary column to represent if the prediction was correct.
test$predicted_correct <- ifelse(test$category == test$predicted_category, 1, 0)

test_num_correct_predictions <- sum(test$predicted_correct)
svm_3_test_prop_predicted_correctly <- test_num_correct_predictions/test_observations

svm_3_test_prop_predicted_correctly


Build a model using h2o auto ML library:


Initiate auto ML and fit model:

**Be mindful that this chunk will take approximately 10 minutes to run:**


In [ ]:
knitr::opts_chunk$set(cache = TRUE)

h2o.init()
h2o.no_progress()

lagged_dataset$category <- as.factor(lagged_dataset$category)

h20.data <- as.h2o(lagged_dataset[,c(2,9,11, 18, 19)])

auto_ml_test <- as.h2o(select(test, Open, qav, volatility, profit), use_datatable = TRUE)

auto_ml_full <- as.h2o(select(data, Open, qav, volatility, profit), use_datatable = TRUE)

aml <- h2o.automl(x = c("Open", "qav", "volatility", "profit"),
                  y = "category",
                  training_frame = h20.data,
                  max_runtime_secs = 500)

aml@leaderboard
h2o.performance(aml@leader)

#save best model
best_model <- h2o.get_best_model(aml, "any")

#display best autoML model
#aml@leader
#aml@leader@allparameters

#obtain the predictions
h2o_preds <- predict(best_model, newdata = auto_ml_test)
h2o_preds_df <- as.data.frame(h2o_preds)

h2o_preds_full <- predict(best_model, newdata = auto_ml_full)
h2o_preds_df_full <- as.data.frame(h2o_preds_full)

data$auto_ml_predicted_category <- NA
data$auto_ml_predicted_category <- h2o_preds_df_full$predict



Calculate prediction error:


In [ ]:
test_observations <- count(test)$n

test$auto_ml_prediction <- h2o_preds_df$predict

#Calculate the number of correct predictions
#make a binary column to represent if the prediction was correct.
test$auto_ml_predicted_correct <- ifelse(test$category == test$auto_ml_prediction, 1, 0)

num_correct_predictions <- sum(test$auto_ml_predicted_correct)
auto_ml_prop_predicted_correctly <- num_correct_predictions/test_observations

auto_ml_prop_predicted_correctly


Bar graph of prediction accuracies by model:



In [ ]:
#create a df to hold model names and pred accuracies
model_names <- c("lda","svm1", "svm2", "svm3", "Deep Learning (autoML)")
prediction_accuracies <- c(lda_test_prop_predicted_correctly, svm_1_test_prop_predicted_correctly,
                           svm_2_test_prop_predicted_correctly, svm_3_test_prop_predicted_correctly,
                           auto_ml_prop_predicted_correctly)

tmp2 <- data.frame(cbind(model_names, prediction_accuracies))

#round pred accuracy to 2 digits to make graph easier to read

tmp2$prediction_accuracies <- as.double(tmp2$prediction_accuracies)
tmp2$prediction_accuracies <- round(tmp2$prediction_accuracies, digits = 3)

ggplot(tmp2, aes(x = model_names, y = prediction_accuracies, fill = prediction_accuracies)) + geom_bar(stat = "identity") + 
  geom_text(aes(label = prediction_accuracies, vjust=0)) +
  labs(y = "Prediction Accuracies", x = "Model Name") +
  ggtitle("Prediction Accuracy by Model") + guides(fill=guide_legend(title="Prediction Accuracy"))


The model with the highest prediction accuracy was the SVM3 model (94.2%). Lets backtest the amount of predicted profit and percentage return assuming 0.0027 BTC (equal to 100 USD on 1/30/21) traded per window. I will tell the algorithm to not make trades during intervals where the category predicted was hit_stop. We will also calculate what the "perfect" return could be if everything were classified correctly:


**Predicted Percent Return by Model**

Calculate what the percent return and profit would be in the case of 100% classification accuracy:


In [ ]:
amount_btc_invested <- 0.0027
amount_USD_invested <- 100 * total_observations 

for(i in 1:length(data$category)){
  if(data$category[i] == "HIT_STOP"){
    #if you are predicting it will hit stop then you will just not make the trade, so set the profit to zero
    data$profit[i] <- 0
  }else if(data$category[i] == "TAKE_PROFIT"){
    data$profit[i] <- data$take_profit[i] - data$Open[i]
  }else if(data$category[i] == "HIT_BOTH"){
    data$profit[i] <- data$Close[i] - data$Open[i]
  }else{ 
    #corresponds to BTC price staying within band
    data$profit[i] <- data$Close[i] - data$Open[i]
  }
}

actual_profit <- sum(data$profit)

actual_percent_return <- actual_profit / amount_USD_invested



Calculate predicted profit using SV3 model:


In [ ]:
#obtain the profit and percent return predictions

data$predicted_category <- predict(svm3, data)

#since everything was within the using this model band, trades would close at the closing price

data$predicted_profit <- data$Close -  data$Open
predicted_profit <- sum(data$predicted_profit)

svm_percent_return <- predicted_profit / amount_USD_invested


Calculate percent return using autoML model:



In [ ]:
#Using automl

data$auto_ml_predicted_profit <- ifelse(data$auto_ml_predicted_category == "TAKE_PROFIT",
                                        data$take_profit - data$Open, data$Close - data$Open)

auto_ml_predicted_profit <- sum(data$auto_ml_predicted_profit)

auto_ml_percent_return <-  auto_ml_predicted_profit / amount_USD_invested


**Why I ultimately decided to pick LDA model**

Originally, I was going to submit SVM as the best model. SVM and the deep learning model from autoML have higher predictive accuracy than LDA, but they have major shortcomings. They are achieving high classification accuracy by classifying everything as falling within the band, and are giving high rates of error in predicting the other categories. Out of all 5576 observations in the data set, 5246 actually fall within the trading band, which is how SVM is achieving such high classification accuracy. AutoML is a little bit better and correctly predicts 73/172 cases as hitting the stop loss.


In [ ]:
table(data$category)
table(data$svm_predicted_category)
table(data$auto_ml_predicted_category)


However, incorrectly predicting hitting the stop loss is a big problem. Approximately half of all trading windows have closing prices lower than opening prices, and when predicting, hitting the stop loss is the only instance in which we can remove instances where profit is lost. This is because we can simply instruct the computer not to make trades where stop losses would be hit. Conversely, when we predict that the take profit price will be hit, this is a way to guarantee profit:



In [ ]:
hist(data$profit, main = "Difference Between Closing and Opening Price", xlab = "Difference",
     labels = TRUE)


Ultimately, I decided to choose the LDA model as the final model, and tried to use cross validation to improve its final predictive accuracy. I choose 5 folds instead of 10 because I figured that we didn't have a large enough dataset for 10 folds, but using 5 or 10 did not end up changing the predictive accuracy on the test set. 



In [ ]:
train_control <- trainControl(method="cv", number=5)

lda_cv <- train(category ~ qav + Open + volatility + profit, 
                data = data, 
                method="lda",
                trControl = train_control)
#Generate predictions
data$lda_predicted_category <- predict(lda_cv, newdata = data)
num_observations <- count(data)$n

data$predicted_correct <- ifelse(data$category == data$lda_predicted_category, 1, 0)

num_correct_predictions <- sum(data$predicted_correct)
lda_prop_predicted_correctly <- num_correct_predictions/num_observations

lda_prop_predicted_correctly


#calculate test set error
#Generate predictions
test$predicted_category <- predict(lda_cv, newdata = test)

num_test_observations <- count(test)$n

#Calculate the number of correct predictions
#make a binary column to represent if the prediction was correct.
test$predicted_correct <- ifelse(test$category == test$predicted_category, 1, 0)

num_correct_predictions <- sum(test$predicted_correct)
lda_cv_test_prop_predicted_correctly <- num_correct_predictions/num_test_observations

lda_cv_test_prop_predicted_correctly


Cross validation improved LDA predictve accuracy on the test set from 0.9478886 to 0.9667565:



In [ ]:
print("Predictive accuracy test/train split: ")
lda_test_prop_predicted_correctly

print("Predictive accuracy with 5 fold cv: ")
lda_cv_test_prop_predicted_correctly

table(data$lda_predicted_category)


We see that LDA predicted that 81 rows would be in the stop loss category. Allowing all these trades to be cancelled will save us lots of money. I will calculate the predicted percent return and profit of the LDA model below:



In [ ]:
#predict profit using LDA

data$lda_predicted_profit <- NA

for(i in 1:length(data$lda_predicted_category)){
  if(data$lda_predicted_category[i] == "HIT_STOP"){
    #if you are predicting it will hit stop then you will just not make the trade, so set the profit to zero
    data$lda_predicted_profit[i] <- 0
  }else if(data$lda_predicted_category[i] == "TAKE_PROFIT"){
    data$lda_predicted_profit[i] <- data$take_profit[i] - data$Open[i]
  }else if(data$lda_predicted_category[i] == "HIT_BOTH"){
    data$lda_predicted_profit[i] <- data$Close[i] - data$Open[i]
  }else{ 
    #corresponds to BTC price staying within band
    data$lda_predicted_profit[i] <- data$Close[i] - data$Open[i]
  }
}

lda_predicted_profit <- sum(data$lda_predicted_profit)

lda_percent_return <- lda_predicted_profit / amount_USD_invested


To conclude, the best model actually turned out to be LDA. Initially I ingored this model, but once I used cross validation to fit it, it turned out to have the highest prediction accuracy and the closest percent return to what the best possible percent return could have been with 100% predictive accuracy:



In [ ]:
models <- c("SVM", "Deep Learning", "LDA", "Actual Data")
pct_return <- c(svm_percent_return, auto_ml_percent_return, lda_percent_return, actual_percent_return)

tmp3 <- data.frame(cbind(models, pct_return))
tmp3$pct_return <- as.double(tmp3$pct_return)
tmp3$pct_return <- round(tmp3$pct_return, digits = 2)

ggplot(tmp3, aes(x = models, y = pct_return, fill = models)) +
  geom_bar(stat = "identity") + geom_text(aes(label = pct_return, vjust=0)) + 
  ggtitle("Predicted Percentage Return by Model") + 
  labs(y = "Percentage Return")



To further improve on this method, I would suggest not classifying into categories. Using classification is less helpful for this specific dataset as 94% of the values will fall within the volatility band, meaning you would exit the trade at the closing price. Since approximately half of the time the closing price is lower than the opening, this means we would be losing money on approximately half of the trades. Maybe through predicting a continuous value like the BTC price we would be able to avoid losing money on a higher proportion of trades. This is not guaranteed to work, but rather an idea I had for improving our return. 
